## Librerie

In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

# Range ammissibile percentuale materiale per ricetta
#           | Materiale1        | Materiale2 | ...
# ------------------------------------------------
# Ricetta1  | (val att. ,range) |            |
# Ricetta2  |                   |            |
# ...
range_ric_mat = np.array([
    [(0.58, 0.01), (0.396, 0.003), (0.024, 0.001)], # ricetta 0
    [(0.625, 0.005), (None, None), (None, None)],
    [(0.62, 0.01), (None, None), (None, None)],
])

In [3]:
%run ./importazione.ipynb

(10,)
(28,)
(28, 10)
(28, 13)
(10, 13)
(10, 13)
(10, 13)


In [4]:
print('Verifica dimensioni matrici')
print(f'Consumi (num. famiglie)): {consumi.shape}')
print(f'Produzioni (num. ricette): {produzioni.shape}')
print(f'Ricette (famiglie x ricette): {ricette.shape}')
print(f'Composizioni (famiglie x materiali): {composizioni_famiglia.shape}')
print(f'Range (ricette x materiali): {range_ric_mat.shape}')

Verifica dimensioni matrici
Consumi (num. famiglie)): (28,)
Produzioni (num. ricette): (10,)
Ricette (famiglie x ricette): (280,)
Composizioni (famiglie x materiali): (364,)
Range (ricette x materiali): (10, 13)


## Calcolo resa globale

In [5]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0389346559077537)'

## Funzioni di calcolo

In [6]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    return ricetta.reshape(-1, len(produzioni)) * produzioni

In [7]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [8]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sum(np.square(tot_consumi-consumi))
    return tot_err

In [9]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [10]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

## Analisi composizione

In [11]:
# Matrice consumi usando ricetta iniziale
# cons_fam = calc_mat_consumi(ricette)
# pd.DataFrame(cons_fam)

In [12]:
# Reshape della matrice composizioni (famiglia vs materiale)
# compos = composizioni_famiglia.reshape(len(consumi), -1)
# pd.DataFrame(compos)

In [13]:
# Devo calcolare quanto materiale per ciascuna ricetta

# Proviamo per prima colonna matrice consumi (quindi prima ricetta)
# Moltiplico la colonna consumi di quella ricetta per le singole colonne della matrice composizione

# cons_ricetta0 = np.vstack(cons_fam[:,0])
# print(cons_ricetta0)
# print('')
# cons_materiali_ricetta0 = cons_ricetta0 * compos
# print(cons_materiali_ricetta0)
# print('')
# tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
# print(tot_cons_ricetta0)
# print('')
# print('Composizione ricetta 0')
# tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0)
# print(tot_perc_ricetta0)

In [14]:
# Percentuali di ciascun materiale in una ricetta

# Verifica dimensioni matrici
# Consumi (num. famiglie)): (28,)
# Produzioni (num. ricette): (10,)
# Ricette (famiglie x ricette): (280,)
# Composizioni (famiglie x materiali): (364,)
# Range (ricette x materiali): (28, 13)

def perc_mat(ricetta, id_ric):
    cons_fam = calc_mat_consumi(ricetta)
    compos = composizioni_famiglia.reshape(len(consumi), -1)
    cons_ricetta0 = np.vstack(cons_fam[:,id_ric])
    cons_materiali_ricetta0 = cons_ricetta0 * compos
    tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
    tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0) if np.sum(tot_cons_ricetta0) != 0 else np.zeros((len(tot_cons_ricetta0),1))
    return tot_perc_ricetta0

In [15]:
# Funzione errore percentuale materiale (obiettivo: >=0)
def err_perc_mat(ricetta, id_ric, id_mat, expected_val, expected_error):
    return expected_error - np.abs(perc_mat(ricetta, id_ric=id_ric)[id_mat] - expected_val)

## Ottimizzazione

In [28]:
%%time

constraints = [
    {'type': 'eq', 'fun': calc_err_totali},
]

for id_ric, ric in enumerate(range_ric_mat):
    for id_mat, mat in enumerate(ric):
        if not np.isnan(mat[0]):
            constr = {'type': 'ineq', 'fun': err_perc_mat, 'args': (id_ric, id_mat, mat[0], mat[1])}
            constraints.append(constr)

# constraints = (
#     {'type': 'eq', 'fun': calc_err_totali},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 0, 0.58, 0.01)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 1, 0.396, 0.003)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 2, 0.024, 0.001)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (1, 0, 0.625, 0.005)},
#     # {'type': 'ineq', 'fun': err_perc_mat, 'args': (2, 0, 0.62, 0.01)},
# )

#DEBUG
# del constraints[5]

print(len(constraints))
# constraints = constraints[0] #DEBUG

# for x in constraints:
#     print(x)

bounds = list(( (0, 1) for x in range(len(produzioni)*len(consumi)) ))

# Forza a zero i valori che nella ricetta originale sono già nulli
for index, value in enumerate(ricette):
    if value == 0:
        bounds[index] = (0,0)

res = minimize(
    calc_error_resa, 
    ricette, 
    # method='trust-constr', 
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    options={'disp': True, 'maxiter':200}
)

print(res)

# res = minimize(
#     calc_error_resa, 
#     res.x, 
#     method='trust-constr', 
#     # method='SLSQP',
#     constraints=constraints,
#     bounds=bounds,
#     options={'disp': True, 'maxiter':20}
# )

131
Iteration limit reached    (Exit mode 9)
            Current function value: 8.085471964143057
            Iterations: 200
            Function evaluations: 15871
            Gradient evaluations: 200
 message: Iteration limit reached
 success: False
  status: 9
     fun: 8.085471964143057
       x: [ 0.000e+00  3.565e-02 ...  0.000e+00  0.000e+00]
     nit: 200
     jac: [       nan  3.388e-01 ...        nan        nan]
    nfev: 15871
    njev: 200
CPU times: total: 3min 7s
Wall time: 5min 32s


In [29]:
# Esperimento
# Proviamo a escludere un constraint alla volta e vediamo se troviamo una situazione
# in cui il problema converge.

# for i in range(1, len(constraints)):
    
#     print(i)
    
#     const_red = constraints[0:i] + constraints[i+1:]

#     res = minimize(
#         calc_error_resa, 
#         ricette, 
#         method='SLSQP',
#         constraints=const_red,
#         bounds=bounds,
#         options={'disp': False, 'maxiter':100}
#     )

#     print(f'{res.success} - {res.message} ({res.nit} iterations)')


## Verifiche

In [30]:
pd.options.display.float_format = '{:.2f}'.format

In [31]:
print("Percentuali aggiustate (in %)")
pd.DataFrame(
    res.x.reshape(len(consumi), len(produzioni))*100
)

Percentuali aggiustate (in %)


,0,1,2,3,4,5,6,7,8,9
0,0.00,3.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,55.89,62.29,32.27,48.45,0.00,0.00,0.00,0.00,39.05,0.00
2,0.00,0.00,42.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,42.93,0.00,0.00,0.00,0.02,0.00
4,0.00,0.00,0.00,0.00,0.00,5.18,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,45.76,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.72
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.71,0.00
8,39.11,35.11,28.34,24.79,0.00,0.00,0.00,0.00,34.74,0.00
9,0.00,0.00,0.03,0.00,63.67,0.59,0.00,0.00,0.00,0.00


In [32]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

Matrice consumi


,0,1,2,3,4,5,6,7,8,9
0,0.00,1247486.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,3240301.46,21796171.78,2413741.84,45183341.94,0.00,0.00,0.00,0.00,1957609.50,0.00
2,0.00,0.00,3147918.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,1610592.05,0.00,0.00,0.00,1093.45,0.00
4,0.00,0.00,0.00,0.00,0.00,70343.44,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,718523.87,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22061.09
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,336297.33,0.00
8,2267551.89,12287496.41,2119803.78,23117674.72,0.00,0.00,0.00,0.00,1741487.68,0.00
9,0.00,0.00,1883.56,0.00,2388848.02,7967.72,0.00,0.00,0.00,0.00


In [33]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

Verifica totale consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1247486.18,74591166.52,3147918.99,1611685.50,70343.44,718523.87,22061.09,336297.33,41534014.48,2398699.30,...,10409645.93,1450456.15,57935.73,9483.30,4670.41,13099.83,28354.11,1772.90,10875.46,11637.77


In [34]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x)) - consumi])

Verifica errore consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,104.18,-121.21,-8.89,4038.90,79.94,246.57,-1565.91,77.33,-44.86,3810.30,...,-561.07,-35770.85,53.73,7133.30,228.41,894.83,4027.11,1559.90,4479.96,794.77


In [35]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

Verifica rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,1.37,1.21,1.25,0.81,3.83,1.14,1.06,1.13,1.21,0.83


In [36]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

Verifica errore rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,0.33,0.17,0.21,-0.22,2.79,0.10,0.02,0.09,0.17,-0.21


In [37]:
print('Percentuali materiali ricetta 0')
print(pd.DataFrame(range_ric_mat[0])[['f0']])
pd.DataFrame([perc_mat(res.x, 0) * 100])

Percentuali materiali ricetta 0
     f0
0  0.59
1  0.02
2  0.00
3  0.00
4  0.00
5  0.00
6  0.00
7  0.00
8  0.00
9  0.00
10 0.00
11 0.00
12 0.38


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,60.57,4.68,0.12,0.13,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.46


In [38]:
print('Percentuali materiali ricetta 1')
print(range_ric_mat[1])
pd.DataFrame([perc_mat(res.x, 1) * 100])

Percentuali materiali ricetta 1
[(5.80000e-01, 0.1) (2.90000e-02, 0.1) (1.65000e-03, 0.1)
 (1.65000e-03, 0.1) (1.00000e-03, 0.1) (1.50000e-04, 0.1)
 (1.00000e-04, 0.1) (5.00000e-05, 0.1) (1.00000e-04, 0.1)
 (7.50000e-05, 0.1) (5.00000e-05, 0.1) (0.00000e+00, 0.1)
 (3.86175e-01, 0.1)]


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,58.11,6.54,0.15,0.16,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.97


In [39]:
print('Percentuali materiali ricetta 2')
print(range_ric_mat[2])
pd.DataFrame([perc_mat(res.x, 2) * 100])

Percentuali materiali ricetta 2
[(6.18000e-01, 0.1) (2.00000e-02, 0.1) (6.50000e-04, 0.1)
 (6.50000e-04, 0.1) (5.00000e-04, 0.1) (1.50000e-04, 0.1)
 (1.00000e-04, 0.1) (5.00000e-05, 0.1) (1.00000e-03, 0.1)
 (7.50000e-05, 0.1) (5.00000e-05, 0.1) (0.00000e+00, 0.1)
 (3.58775e-01, 0.1)]


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,59.06,1.50,0.11,0.13,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,39.16
